In [3]:
from transformers import pipeline
summarizer = pipeline("summarization")
classifier = pipeline("text-classification")
ner_tagger = pipeline("ner", aggregation_strategy="simple")
translator = pipeline("translation_en_to_sv", model="Helsinki-NLP/opus-mt-en-sv") #This one you can change to other language then swedish if you like!
generator = pipeline("text-generation")
qa_pipeline = pipeline("question-answering")

print(translator("I love programming in Python"))
print(generator("I love programming in Python"))
print(classifier("I love programming in Python"))
print(ner_tagger("I love programming in Python"))
print(summarizer("I love programming in Python"))

swedish_game_review = """Att följa upp en succé är inte alltid det enklaste, men i fallet God of War: Ragnarök känns det som om Santa Monica Studio trivs som allra bäst när förväntningarna är som störst. Vi har betygsatt Kratos senaste kapitel, och det här är sannerligen en saga du inte vill missa."""

english_game_review = """Santa Monica Studios' follow-up to the 2018 soft reboot is almost here, but does it stack up to its excellent predecessor? Simply put... yes."""

swedish_sports_news = """Det är inte bara på fotbollsplanen som livet leker för Jennifer Falk just nu. Efter succén i Häcken och stora möjligheter att ta över som förstamålvakt i landslaget efter Hedvig Lindahl har hon nu fått livets största gåva."""

celebrity_news = """Television reality star Kim Kardashian West said Friday that she and rapper Kanye West have welcomed their fourth child, a boy born via surrogate.

"He's here and he's perfect!" the 38-year-old Kardashian, who also has her own make-up and fragrance lines, tweeted.

She had announced the couple were expecting in January, telling the US program "Watch What Happens Live With Andy Cohen" that the youngest Kardashian to keep up with was due "sometime soon."

The new baby joins three other siblings: sister Chicago, who was born last year via surrogate, five-year-old sister North, and three-year-old brother Saint."""


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.poole

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.

In [ ]:
texts = [
    swedish_game_review,
    english_game_review,
    swedish_sports_news,
    celebrity_news
]

print("Sentiment classification results:")
for i, text in enumerate(texts, 1):
    result = classifier(text)
    print(f"\nText {i}:")
    print(f"Label: {result[0]['label']}")
    print(f"Score: {result[0]['score']:.4f}")



Sentiment classification results:

Text 1:
Label: NEGATIVE
Score: 0.9883

Text 2:
Label: POSITIVE
Score: 0.9910

Text 3:
Label: NEGATIVE
Score: 0.9893

Text 4:
Label: POSITIVE
Score: 0.9996


In [ ]:
qa_tasks = [
    {
        "context": english_game_review,
        "questions": [
            "When was the predecessor released?",
            "Was the last game good?"
        ]
    },
    {
        "context": celebrity_news,
        "questions": [
            "How many children does kanye have?",
            "How old are Kim Karashian?",
            "How old is North?"
        ]
    }
]


for task in qa_tasks:
    context = task["context"]
    print(f"\nContext: {context[:50]}...")
    
    for question in task["questions"]:
        result = qa_pipeline(question=question, context=context)
        print(f"\nQuestion: {question}")
        print(f"Answer: {result['answer']}")
        print(f"Confidence: {result['score']:.4f}")

Question Answering Results:

Context: Santa Monica Studios' follow-up to the 2018 soft r...

Question: When was the predecessor released?
Answer: 2018
Confidence: 0.9877

Question: Was the last game good?
Answer: does it stack up to its excellent predecessor
Confidence: 0.2574

Context: Television reality star Kim Kardashian West said F...

Question: How many children does kanye have?
Answer: fourth
Confidence: 0.6514

Question: How old are Kim Karashian?
Answer: 38
Confidence: 0.9386

Question: How old is North?
Answer: 38
Confidence: 0.2555

Now try asking your own questions to see how well the model performs!


In [ ]:
def summarize_and_translate(text, max_length=50):
    summary = summarizer(text, max_length=max_length, min_length=10, do_sample=False)[0]['summary_text']
    compression_ratio = len(summary.split()) / len(text.split())
    translation = translator(summary)[0]['translation_text']
    return summary, compression_ratio, translation

for i, text in enumerate(texts, 1):
    print(f"\nText {i}:")
    print(f"Original length: {len(text.split())} words")
    
    for max_length in [30, 50, 70]:
        summary, compression_ratio, translation = summarize_and_translate(text, max_length)
        
        print(f"\nMax length: {max_length}")
        print(f"Summary: {summary}")
        print(f"Summary length: {len(summary.split())} words")
        print(f"Compression ratio: {compression_ratio:.2f}")
        print(f"Swedish translation: {translation}")

    print("\n" + "-"*50)



Text 1:
Original length: 50 words

Max length: 30
Summary:  Santa Monica Studio trivs som allra bäst när förväntningarna är som stör
Summary length: 12 words
Compression ratio: 0.24
Swedish translation: Santa Monica Studio trivs som alla bästa när förbrukningarna är som gör

Max length: 50
Summary:  God of War: Ragnarök känns det som om om Santa Monica Studio trivs som allra bäst när förväntningarna är som störst .
Summary length: 22 words
Compression ratio: 0.44
Swedish translation: God of War: Ragnarök känner det som om Santa Monica Studio trivs som alla bästa när förbrukningarna är som störst.

Max length: 70
Summary:  God of War: Ragnarök känns det som om om Santa Monica Studio trivs som allra bäst när förväntningarna är som störst .
Summary length: 22 words
Compression ratio: 0.44
Swedish translation: God of War: Ragnarök känner det som om Santa Monica Studio trivs som alla bästa när förbrukningarna är som störst.

--------------------------------------------------

Text 2:
Origi

Your max_length is set to 50, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)



Max length: 30
Summary:  Santa Monica Studios' follow-up to the 2018 soft reboot is almost here, but does it stack up to its excellent predecessor? Simply
Summary length: 22 words
Compression ratio: 0.92
Swedish translation: Santa Monica Studios uppföljning av den mjuka omstarten 2018 är nästan här, men staplar den upp till sin utmärkta föregångare?


Your max_length is set to 70, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)



Max length: 50
Summary:  Santa Monica Studios' follow-up to the 2018 soft reboot is almost here, but does it stack up to its excellent predecessor? Simply put, yes. Santa Monica studies have a sequel coming out this summer .
Summary length: 35 words
Compression ratio: 1.46
Swedish translation: Santa Monica Studios uppföljning av 2018 års mjuka omstart är nästan här, men staplar den upp till sin utmärkta föregångare? Enkelt uttryckt, ja. Santa Monica studier har en uppföljare som kommer ut i sommar.

Max length: 70
Summary:  Santa Monica Studios' follow-up to the 2018 soft reboot is almost here, but does it stack up to its excellent predecessor? Simply put, yes. Santa Monica studies have a sequel coming out this summer .
Summary length: 35 words
Compression ratio: 1.46
Swedish translation: Santa Monica Studios uppföljning av 2018 års mjuka omstart är nästan här, men staplar den upp till sin utmärkta föregångare? Enkelt uttryckt, ja. Santa Monica studier har en uppföljare som kommer ut 

# I'll skip the fake generating tweets

I feel like everyone has already tried this since LLMs are more generally available.